In [7]:
import random
import pandas as pd
import numpy as np
import math
from operator import itemgetter
import argparse
np.random.seed(1024)

K=3
similarityMeasure="pearson"
data_file="../data/slopeOne-jupyter.txt"



In [8]:
data_fields = ['user_id', 'item_id', 'rating']
data = pd.read_table(data_file, names=data_fields)
# 二维字典
train_data = {}

# 按照1:9划分数据集
for (user, item,record) in data.itertuples(index=False):
    train_data.setdefault(user,{})
    train_data[user][item] = record

n_users = len(set(data['user_id'].values))
n_items = len(set(data['item_id'].values))

train_data

{'A': {'a': 5, 'b': 10, 'c': 10, 'd': 5},
 'B': {'a': 4, 'b': 5, 'c': 4, 'd': 10},
 'C': {'a': 4, 'c': 10, 'd': 5}}

In [9]:
# 建立item_user倒排表
item_users = dict()
for u, items in train_data.items():
    for i in items:
        if i not in item_users:
            item_users[i] = set()
        item_users[i].add(u)
item_users

{'a': {'A', 'B', 'C'},
 'b': {'A', 'B'},
 'c': {'A', 'B', 'C'},
 'd': {'A', 'B', 'C'}}

In [10]:
def predict(user, j):
    rating = 0
    cnt = 0
    if j not in item_users:
        return rating
    for i in train_data[user]:
        if i == j:
            continue
        if i not in item_users:
            continue
        common_users = item_users[i] & item_users[j]
        if len(common_users) == 0:
            continue
        dev_ij = 0
        cnt += 1
        for common_user in common_users:
            dev_ij += (train_data[common_user][j] - train_data[common_user][i])
        dev_ij /= len(common_users)
        rating += (dev_ij + train_data[user][i])
    rating /= cnt
    return rating


In [11]:
predict('C','b')

7.5